In [1]:
#Library Imports
import numpy as np
import pandas as pd
import math
import os
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error


from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold

In [39]:
train=pd.read_csv('train.csv', encoding = "euc-kr")
test=pd.read_csv('test.csv', encoding = "euc-kr")


In [40]:
#train.shape 122400 X 10
#60개의 건물 X 85일 24시간 =122400
train

,num,date_time,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유
0,1,2020-06-01 00,8179.056,17.6,2.5,92.0,0.8,0.0,0.0,0.0
1,1,2020-06-01 01,8135.640,17.7,2.9,91.0,0.3,0.0,0.0,0.0
2,1,2020-06-01 02,8107.128,17.5,3.2,91.0,0.0,0.0,0.0,0.0
3,1,2020-06-01 03,8048.808,17.1,3.2,91.0,0.0,0.0,0.0,0.0
4,1,2020-06-01 04,8043.624,17.0,3.3,92.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
122395,60,2020-08-24 19,4114.368,27.8,2.3,68.0,0.0,0.7,1.0,1.0
122396,60,2020-08-24 20,3975.696,27.3,1.2,71.0,0.0,0.0,1.0,1.0
122397,60,2020-08-24 21,3572.208,27.3,1.8,71.0,0.0,0.0,1.0,1.0
122398,60,2020-08-24 22,3299.184,27.1,1.8,74.0,0.0,0.0,1.0,1.0


In [41]:
#test.shape 10080 X 9
#60개의 건물 X 7일 24시간 =10080
test

,num,date_time,기온(°C),풍속(m/s),습도(%),"강수량(mm, 6시간)","일조(hr, 3시간)",비전기냉방설비운영,태양광보유
0,1,2020-08-25 00,27.8,1.5,74.0,0.0,0.0,NaN,NaN
1,1,2020-08-25 01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2020-08-25 02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2020-08-25 03,27.3,1.1,78.0,NaN,0.0,NaN,NaN
4,1,2020-08-25 04,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
10075,60,2020-08-31 19,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10076,60,2020-08-31 20,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10077,60,2020-08-31 21,27.9,4.1,68.0,NaN,0.0,1.0,1.0
10078,60,2020-08-31 22,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# **데이터 전처리**

In [42]:
#건물별로 '비전기냉방설비운영'과 '태양광보유'를 판단해 test set의 결측치를 보간해줍니다
train[['num', '비전기냉방설비운영','태양광보유']]
ice={}
hot={}
count=0
for i in range(0, len(train), len(train)//60):
    count +=1
    ice[count]=train.loc[i,'비전기냉방설비운영']
    hot[count]=train.loc[i,'태양광보유']
    

In [43]:
for i in range(len(test)):
    test.loc[i, '비전기냉방설비운영']=ice[test['num'][i]]
    test.loc[i, '태양광보유']=hot[test['num'][i]]

In [44]:
#시간 변수와 요일 변수를 추가해봅니다.
def time(x):
    return int(x[-2:])
train['time']=train['date_time'].apply(lambda x: time(x))
test['time']=test['date_time'].apply(lambda x: time(x))

def weekday(x):
    return pd.to_datetime(x[:10]).weekday()
train['weekday']=train['date_time'].apply(lambda x :weekday(x))
test['weekday']=test['date_time'].apply(lambda x :weekday(x))

+ test 결측치 보간해줍니다.
+ test 데이터의 변수는 예보 데이터이며, 예보 데이터는 train 데이터의 기간에 생성된 것이기에 활용 가능합니다.

In [45]:
test = test.interpolate(method='values')

In [46]:
# train['불쾌지수'] = 1.8*train['기온(°C)'] - 0.55*(1-(train['습도(%)']/100))*(1.8*train['기온(°C)']-26) + 32
# test['불쾌지수'] = 1.8*test['기온(°C)'] - 0.55*(1-(test['습도(%)']/100))*(1.8*test['기온(°C)']-26) + 32

# # (데이터프레임 변수 순서 정리)
# train = train[['num','date_time', 'time', 'weekday','기온(°C)','습도(%)','불쾌지수','풍속(m/s)','강수량(mm)','일조(hr)','비전기냉방설비운영','태양광보유','전력사용량(kWh)']]
# test = test[['num','date_time', 'time', 'weekday','기온(°C)','습도(%)','불쾌지수','풍속(m/s)','강수량(mm)','일조(hr)','비전기냉방설비운영','태양광보유','전력사용량(kWh)' ]]


In [47]:
# test_13 = test['num'] == 13
# test = test[test_13]
# test

In [48]:
# train_13 = train['num'] == 13
# train = train[train_13]
# train

In [49]:
# test_y = test['전력사용량(kWh)']
# test_x = test.drop(['전력사용량(kWh)'], axis = 1)
test_x = test

# **모델링**

In [50]:
test_x

,num,date_time,기온(°C),풍속(m/s),습도(%),"강수량(mm, 6시간)","일조(hr, 3시간)",비전기냉방설비운영,태양광보유,time,weekday
0,1,2020-08-25 00,27.800000,1.500000,74.000000,0.0,0.000000,0.0,0.0,0,1
1,1,2020-08-25 01,27.633333,1.366667,75.333333,0.0,0.000000,0.0,0.0,1,1
2,1,2020-08-25 02,27.466667,1.233333,76.666667,0.0,0.000000,0.0,0.0,2,1
3,1,2020-08-25 03,27.300000,1.100000,78.000000,0.0,0.000000,0.0,0.0,3,1
4,1,2020-08-25 04,26.900000,1.166667,79.666667,0.0,0.000000,0.0,0.0,4,1
...,...,...,...,...,...,...,...,...,...,...,...
10075,60,2020-08-31 19,28.633333,3.566667,66.000000,0.0,0.533333,1.0,1.0,19,0
10076,60,2020-08-31 20,28.266667,3.833333,67.000000,0.0,0.266667,1.0,1.0,20,0
10077,60,2020-08-31 21,27.900000,4.100000,68.000000,0.0,0.000000,1.0,1.0,21,0
10078,60,2020-08-31 22,27.900000,4.100000,68.000000,0.0,0.000000,1.0,1.0,22,0


In [51]:
train_x=train.drop('전력사용량(kWh)', axis=1)
train_y=train[['전력사용량(kWh)']]

In [52]:
train_x.drop('date_time', axis=1, inplace=True)
test_x.drop('date_time', axis=1, inplace=True)

In [53]:
# minmax_scaler = MinMaxScaler()
# train_x = minmax_scaler.fit_transform(train_x)
# test_x = minmax_scaler.fit_transform(test_x)

In [54]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import VotingRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.pipeline import make_pipeline

In [55]:
# from sklearn.linear_model import LinearRegression
# lr = LinearRegression(n_jobs=-1)
# lr.fit(train_x, train_y)
# lr_pred = lr.predict(test_x)

In [56]:
poly_pipeline = make_pipeline(
    PolynomialFeatures(degree=2, include_bias=False),
    StandardScaler(),
    ElasticNet(alpha=0.1, l1_ratio=0.2,random_state=42)
)
poly_pred = poly_pipeline.fit(train_x, train_y).predict(test_x)

In [ ]:
# #랜덤포레스트 하이퍼파라매터튜닝 v-> 쓸모없어


# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import GridSearchCV
# params = {
#     'n_estimators': [300, 100, 500, 1000], 
#     'max_depth': [3, 5, None, 7], 
#     'min_samples_split': [2, 3, 4, 5], 
#     'max_features': ['auto', 'log2', 'sqrt', 0.9, 0.8]
# }
# grid_search = GridSearchCV(RandomForestRegressor(), params, cv=3, n_jobs=-1, scoring='neg_mean_squared_error')
# grid_search.fit(train_x, train_y)
# grid_search.best_params_

In [57]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(random_state=42, n_estimators=300, max_depth=5, max_features=0.9)
rfr.fit(train_x, train_y)
rfr_pred = rfr.predict(test_x)

<ipython-input-57-b9bfaa157188>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfr.fit(train_x, train_y)


In [58]:
#  #GBR 하이퍼파라매터튜닝
# params_gbr = {
#     'n_estimators': [10, 100, 200, 1000], 
#     'learning_rate': [0.1, 0.01, 0.005, 0.001], 
#     'min_samples_split': [2, 3, 4], 
#     'subsample': [0.2, 0.3, 0.1, 0.5]
# }
# grid_search = GridSearchCV(GradientBoostingRegressor(), params_gbr, cv=3, n_jobs=-1, scoring='neg_mean_squared_error')
# grid_search.fit(train_x, train_y)
# grid_search.best_params_

In [59]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor(random_state=42,learning_rate = 0.01, n_estimators = 100, subsample = 0.45)
gbr.fit(train_x, train_y)
gbr_pred = gbr.predict(test_x)

/anaconda/envs/py38_tensorflow/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [60]:
# !pip install lightgbm

In [61]:
# from lightgbm import LGBMRegressor
# lgbm = LGBMRegressor(random_state=42)
# lgbm.fit(train_x, train_y)
# lgbm_pred = lgbm.predict(test_x)


In [62]:
!pip install xgboost

In [63]:
from xgboost import XGBRegressor
xgb = XGBRegressor(random_state=42, learning_rate = 0.1, n_estimators = 1000, subsample = 0.45)
xgb.fit(train_x, train_y)
xgb_pred = xgb.predict(test_x)

In [64]:
from sklearn.ensemble import StackingRegressor
stack_models = [
    ('elasticnet', poly_pipeline), 
    ('randomforest', rfr), 
    ('gbr', gbr),
] 

In [65]:
stack_reg = StackingRegressor(stack_models, final_estimator=xgb, n_jobs=-1)
stack_reg.fit(train_x, train_y)
stack_pred = stack_reg.predict(test_x)

/anaconda/envs/py38_tensorflow/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [66]:
#weighted blending
final_outputs = {
    'elasticnet': poly_pred, 
    'randomforest': rfr_pred, 
    'gbr': gbr_pred,
    'xgb': xgb_pred,
    'stacking': stack_pred,
}

In [67]:
final_prediction=\
final_outputs['elasticnet'] * 0.20\
+final_outputs['randomforest'] * 0.60\
+final_outputs['gbr'] * 0.05\
+final_outputs['xgb'] * 0.05\
+final_outputs['stacking'] * 0.10


In [68]:
final_prediction

array([6733.39832815, 6749.60537932, 6718.04954145, ..., 3221.84889464,
       3148.01031111, 3178.70660474])

In [69]:


def SMAPE(true, pred):
    return np.mean((np.abs(true-pred))*2/(np.abs(true) + np.abs(pred)))*100


In [70]:
test_y

,num,date_time,기온(°C),풍속(m/s),습도(%),"강수량(mm, 6시간)","일조(hr, 3시간)",비전기냉방설비운영,태양광보유,time,weekday
0,1,2020-08-25 00,27.8,1.5,74.0,0.0,0.0,0.0,0.0,0,1
1,1,2020-08-25 01,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1,1
2,1,2020-08-25 02,NaN,NaN,NaN,NaN,NaN,0.0,0.0,2,1
3,1,2020-08-25 03,27.3,1.1,78.0,NaN,0.0,0.0,0.0,3,1
4,1,2020-08-25 04,NaN,NaN,NaN,NaN,NaN,0.0,0.0,4,1
...,...,...,...,...,...,...,...,...,...,...,...
10075,60,2020-08-31 19,NaN,NaN,NaN,NaN,NaN,1.0,1.0,19,0
10076,60,2020-08-31 20,NaN,NaN,NaN,NaN,NaN,1.0,1.0,20,0
10077,60,2020-08-31 21,27.9,4.1,68.0,NaN,0.0,1.0,1.0,21,0
10078,60,2020-08-31 22,NaN,NaN,NaN,NaN,NaN,1.0,1.0,22,0


In [71]:
stack_pred

array([2494.1223, 2528.1545, 2099.066 , ..., 3382.377 , 2799.3308,
       3252.4937], dtype=float32)

In [77]:
final_prediction.shape

(10080,)

In [100]:
submission=pd.read_csv('sample_submission.csv', encoding = "euc-kr")

In [101]:
submission

,num_date_time,answer
0,1 2020-08-25 00,0
1,1 2020-08-25 01,0
2,1 2020-08-25 02,0
3,1 2020-08-25 03,0
4,1 2020-08-25 04,0
...,...,...
10075,60 2020-08-31 19,0
10076,60 2020-08-31 20,0
10077,60 2020-08-31 21,0
10078,60 2020-08-31 22,0


In [103]:
submission['answer'] = final_prediction

In [104]:
submission

,num_date_time,answer
0,1 2020-08-25 00,6733.398328
1,1 2020-08-25 01,6749.605379
2,1 2020-08-25 02,6718.049541
3,1 2020-08-25 03,6731.336093
4,1 2020-08-25 04,6835.847168
...,...,...
10075,60 2020-08-31 19,3238.334732
10076,60 2020-08-31 20,3338.746325
10077,60 2020-08-31 21,3221.848895
10078,60 2020-08-31 22,3148.010311


In [105]:
submission.to_csv('baseline_submission3.csv', index=False)